In [2]:
#Log when a function starts and ends.
#logging deployments, restarts, backups.

from functools import wraps

def logger(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"[LOG] Starting {func.__name__}")
        result = func(*args, **kwargs)
        print(f"[LOG] Finished {func.__name__}")
        return result
    return wrapper

@logger
def deploy(service):
    print(f"Deploying {service}")

deploy("payment")

[LOG] Starting deploy
Deploying payment
[LOG] Finished deploy


In [4]:
#Retry a failing operation (network, API, SSH)

from functools import wraps
import time

def retry(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        attempts = 3
        for attempt in range(1, attempts + 1):
            try:
                return func(*args, **kwargs)
            except Exception as e:
                print(f"Attempt {attempt} failed: {e}")
                time.sleep(1)
        raise Exception("Max retries exceeded")
    return wrapper

@retry
def connect():
    raise Exception("Network error")

connect()

Attempt 1 failed: Network error
Attempt 2 failed: Network error
Attempt 3 failed: Network error


Exception: Max retries exceeded

In [5]:
#Allow execution only for authorized users
from functools import wraps

def authorize(func):
    @wraps(func)
    def wrapper(user, *args, **kwargs):
        if user != "admin":
            raise PermissionError("Unauthorized access")
        return func(user, *args, **kwargs)
    return wrapper

@authorize
def delete_server(user, server):
    print(f"{server} deleted by {user}")

delete_server("admin", "web01")

web01 deleted by admin


In [6]:
# Customize decorator behavior (e.g., retries count)

from functools import wraps
import time

def retry_with_limit(max_retries):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(1, max_retries + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Retry {attempt}/{max_retries}: {e}")
                    time.sleep(1)
            raise Exception("All retries failed")
        return wrapper
    return decorator

@retry_with_limit(5)
def fetch_data():
    raise Exception("Timeout")

fetch_data()

Retry 1/5: Timeout
Retry 2/5: Timeout
Retry 3/5: Timeout
Retry 4/5: Timeout
Retry 5/5: Timeout


Exception: All retries failed

In [7]:
## Template

from functools import wraps

def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # before
        result = func(*args, **kwargs)
        # after
        return result
    return wrapper
